In [1]:
epochs = 10
n_test_batches = 200

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
import syft as sy
hook = sy.TorchHook(torch) 
client = sy.VirtualWorker(hook, id="client")
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider") 

In [4]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 50
        self.epochs = epochs
        self.lr = 0.001
        self.log_interval = 100

args = Arguments()

In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True)

In [6]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True)

private_test_loader = []
for data, target in test_loader:
    private_test_loader.append((
        data.fix_precision().share(alice, bob, crypto_provider=crypto_provider, protocol="fss"),
        target.fix_precision().share(alice, bob, crypto_provider=crypto_provider, protocol="fss")
    ))


In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [8]:
def train(args, model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        output = F.log_softmax(output, dim=1)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size,
                100. * batch_idx / len(train_loader), loss.item()))

In [9]:
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

for epoch in range(1, args.epochs + 1):
    train(args, model, train_loader, optimizer, epoch)

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.280699
Train Epoch: 1 [6400/60032 (11%)]	Loss: 0.302638
Train Epoch: 1 [12800/60032 (21%)]	Loss: 0.357552
Train Epoch: 1 [19200/60032 (32%)]	Loss: 0.162103
Train Epoch: 1 [25600/60032 (43%)]	Loss: 0.116576
Train Epoch: 1 [32000/60032 (53%)]	Loss: 0.211301
Train Epoch: 1 [38400/60032 (64%)]	Loss: 0.115454
Train Epoch: 1 [44800/60032 (75%)]	Loss: 0.051757
Train Epoch: 1 [51200/60032 (85%)]	Loss: 0.034267
Train Epoch: 1 [57600/60032 (96%)]	Loss: 0.286765
Train Epoch: 2 [0/60032 (0%)]	Loss: 0.023893
Train Epoch: 2 [6400/60032 (11%)]	Loss: 0.062522
Train Epoch: 2 [12800/60032 (21%)]	Loss: 0.062613
Train Epoch: 2 [19200/60032 (32%)]	Loss: 0.090940
Train Epoch: 2 [25600/60032 (43%)]	Loss: 0.062988
Train Epoch: 2 [32000/60032 (53%)]	Loss: 0.070532
Train Epoch: 2 [38400/60032 (64%)]	Loss: 0.016616
Train Epoch: 2 [44800/60032 (75%)]	Loss: 0.037788
Train Epoch: 2 [51200/60032 (85%)]	Loss: 0.004919
Train Epoch: 2 [57600/60032 (96%)]	Loss: 0.011107
Train Epoch:

In [10]:
def test(args, model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            output = F.log_softmax(output, dim=1)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
test(args, model, test_loader)


Test set: Average loss: 0.1027, Accuracy: 9753/10000 (98%)



In [12]:
model.fix_precision().share(alice, bob, crypto_provider=crypto_provider, protocol="fss")

Net(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [13]:
def test(args, model, test_loader):
    model.eval()
    n_correct_priv = 0
    n_total = 0
    with torch.no_grad():
        for data, target in test_loader[:n_test_batches]:
            output = model(data)
            pred = output.argmax(dim=1) 
            n_correct_priv += pred.eq(target.view_as(pred)).sum()
            n_total += args.test_batch_size

            n_correct = n_correct_priv.copy().get().float_precision().long().item()
    
            print('Test set: Accuracy: {}/{} ({:.0f}%)'.format(
                n_correct, n_total,
                100. * n_correct / n_total))

In [14]:
test(args, model, private_test_loader)

Test set: Accuracy: -9476747264/50 (-18953494528%)
Test set: Accuracy: -25386237952/100 (-25386237952%)
Test set: Accuracy: -19945598976/150 (-13297065984%)
Test set: Accuracy: -19945598976/200 (-9972799488%)
Test set: Accuracy: -9771836416/250 (-3908734566%)
Test set: Accuracy: -52721508352/300 (-17573836117%)
Test set: Accuracy: -52721508352/350 (-15063288101%)
Test set: Accuracy: -52127395840/400 (-13031848960%)
Test set: Accuracy: -51503710208/450 (-11445268935%)
Test set: Accuracy: -51503710208/500 (-10300742042%)
Test set: Accuracy: -51503710208/550 (-9364310947%)
Test set: Accuracy: -58668490752/600 (-9778081792%)
Test set: Accuracy: -58668490752/650 (-9025921654%)
Test set: Accuracy: -58668490752/700 (-8381212965%)
Test set: Accuracy: -58668490752/750 (-7822465434%)
Test set: Accuracy: -54498357248/800 (-6812294656%)
Test set: Accuracy: -54498357248/850 (-6411571441%)
Test set: Accuracy: -54498357248/900 (-6055373028%)
Test set: Accuracy: -56317009920/950 (-5928106307%)
Test se